In [ ]:
import requests
import snowflake.connector
from datetime import datetime

def fetch_prices():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin,ethereum&vs_currencies=usd"
    res = requests.get(url)
    data = res.json()

    # Connect to Snowflake
    conn = snowflake.connector.connect(
        user=os.environ['SNOWFLAKE_USER'],
        password=os.environ['SNOWFLAKE_PASSWORD'],
        account=os.environ['SNOWFLAKE_ACCOUNT'],
        ...
    )
    cur = conn.cursor()
    timestamp = datetime.utcnow()

    for coin, value in data.items():
        price = value['usd']
        cur.execute("""
            INSERT INTO RAW.CRYPTO_PRICES (timestamp, coin, usd_price)
            VALUES (%s, %s, %s)
        """, (timestamp, coin, price))

    conn.commit()


In [ ]:
# scripts/fetch_coingecko_prices.py
import os
import requests
import snowflake.connector
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()  # Loads variables from .env

def fetch_prices_to_snowflake():
    url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin,ethereum&vs_currencies=usd"
    response = requests.get(url)
    data = response.json()
    now = datetime.utcnow()

    conn = snowflake.connector.connect(
        user=os.environ["SNOWFLAKE_USER"],
        password=os.environ["SNOWFLAKE_PASSWORD"],
        account=os.environ["SNOWFLAKE_ACCOUNT"],
        role=os.environ["SNOWFLAKE_ROLE"],
        warehouse=os.environ["SNOWFLAKE_WAREHOUSE"],
        database=os.environ["SNOWFLAKE_DATABASE"],
        schema=os.environ["SNOWFLAKE_SCHEMA"]
    )
    cs = conn.cursor()

    # Create table if not exists
    cs.execute("""
        CREATE TABLE IF NOT EXISTS CRYPTO_PRICES (
            coin STRING,
            usd_price FLOAT,
            fetched_at TIMESTAMP
        )
    """)

    for coin, val in data.items():
        cs.execute("""
            INSERT INTO CRYPTO_PRICES (coin, usd_price, fetched_at)
            VALUES (%s, %s, %s)
        """, (coin, val['usd'], now))

    cs.close()
    conn.close()


In [1]:
import requests

# Example: Raw file URL
url = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin,ethereum&vs_currencies=usd"
response = requests.get(url)
response.json()

{'bitcoin': {'usd': 110209}, 'ethereum': {'usd': 4278.06}}